<a href="https://colab.research.google.com/github/praveenjune17/Neural-Machine-Translation-English-Tamil-model/blob/master/skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Neural Machine Translation with Attention

In [3]:
!pip install tensorflow-gpu
!pip install -U tf-nightly-gpu
from __future__ import absolute_import, division, print_function, unicode_literals
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import unicodedata
import re
import numpy as np
import os
import io
import time
import random
from math import log
import tensorflow as tf
tf.enable_eager_execution()

     |████████████████████████████████| 377.0MB 53kB/s 
     |████████████████████████████████| 396.1MB 51kB/s 
     |████████████████████████████████| 3.2MB 39.7MB/s 
     |████████████████████████████████| 501kB 39.4MB/s 
     |████████████████████████████████| 61kB 23.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


## Download and prepare the dataset

#### Inspired by 
https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/text/nmt_with_attention.ipynb

The dataset is downloaded from

a) github.com/achchuthany/En-Ta-Parallel-Corpus

b) github.com/joshua-decoder/indian-parallel-corpora

c) OPUS



### Goal
Convert Tamil sentences to english sentences using a Basic Seq2Seq with attention

In [4]:
# Download the file
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://download.tensorflow.org/data/spa-eng.zip', 
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

#path_to_file = '../content/en_tam_parrallel_text_nodups_sorted_eng_tam_reversed.txt'

2646016/2638744 [==============================] - 0s 0us/step


In [0]:
if not os.path.exists(path_to_file):
  !unzip en_tam_parrallel_text_nodups_sorted_eng_tam_reversed.v1.zip

In [0]:
# # Converts the unicode file to ascii
# def unicode_to_ascii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn')


# def preprocess_sentence(w,language='tamil'):
    
#     if language=='english':
#       w = unicode_to_ascii(w.lower().strip())
#       w = re.sub(r"([?.!,¿])", r" \1 ", w)
#       w = re.sub(r'[" "]+', " ", w)
#       w = re.sub(r"[^a-zA-Z?.'!]+", " ", w)
#     else:
#       w = re.sub(r"([?.!,¿])", r" \1 ", w.strip())
#       w = re.sub(r'[" "]+', " ", w)
#       w = re.sub('[a-zA-Z?.!]+', " ", w) #replace english alphabets in tamil sentence
#     w = w.rstrip().strip()
#     w = '<start> ' + w + ' <end>'
#     return w

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
# en_sentence = u"Everybody hide!"
# ta_sentence = u"எல்லோரும் மறைக்க!"

# print(preprocess_sentence(ta_sentence))
# print(preprocess_sentence(en_sentence,'english'))

In [0]:
# # 1. Remove the accents
# # 2. Clean the sentences
# # 3. Return word pairs in the format: [ENGLISH, TAMIL]
# def create_dataset(path,start,stop):
#   ex1=[]
#   ex2=[]
#   lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

#   for l in lines[start:stop]:
    
#     eng,ta= (l.split('\t'))
#     ex1.append(preprocess_sentence(eng,language='english'))
#     ex2.append(preprocess_sentence(ta,language='tamil'))
    

#   return zip(*list(zip(ex1,ex2)))

In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs

In [0]:
#en,ta  = create_dataset(path_to_file, start=120000,stop=120090)

#print(en[-1])
#print(ta[-1])

In [0]:
def remove_extra(input_tensor,target_tensor,threshold):
  


  new_input_tensor = []
  new_target_tensor=[]
  
  for i,j in zip(input_tensor,target_tensor):
    if (len(i) <= threshold) and (len(j) <= threshold):
      new_input_tensor.append(i)
      new_target_tensor.append(j)
      
    
  return(new_input_tensor,new_target_tensor)

In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
# def tokenize(inp_lang,targ_lang):
#   inp_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
#       filters='')
#   inp_lang_tokenizer.fit_on_texts(inp_lang)

#   ip_tensor = inp_lang_tokenizer.texts_to_sequences(inp_lang)          #convert the words to numbers of varying lengths
  
#   targ_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
#       filters='')
#   targ_lang_tokenizer.fit_on_texts(targ_lang)

#   tar_tensor = targ_lang_tokenizer.texts_to_sequences(targ_lang)          #convert the words to numbers of varying lengths

  

#   return ip_tensor,inp_lang_tokenizer,tar_tensor,targ_lang_tokenizer

In [0]:
# def load_dataset(path, start,stop):
#     # creating cleaned input, output pairs
#     inp_lang,targ_lang  = create_dataset(path, start,stop)
#     input_tensor, inp_lang_tokenizer, target_tensor, targ_lang_tokenizer = tokenize(inp_lang,targ_lang)
#     unk_id = list(inp_lang_tokenizer.word_index.values())[-1] + 1
#     inp_lang_tokenizer.word_index['<UNK>'] = unk_id
#     inp_lang_tokenizer.index_word[unk_id]  = '<UNK>'

#     return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer, unk_id

In [0]:
def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)

    # index language using the class defined above    
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex(en for en, sp in pairs)
    
    # Vectorize the input and target languages
    
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    
    # English sentences
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [0]:
# # Try experimenting with the size of that dataset

# start_lines = 160970
# stop_lines  = 228700
# input_tensor, target_tensor, inp_lang, targ_lang,unk_id = load_dataset(path_to_file, start=start_lines,stop=stop_lines)

# # Calculate max_length of the target tensors
# prev_ip = len(input_tensor)
# prev_tar = len(target_tensor)
# input_tensor,target_tensor = remove_extra(input_tensor,target_tensor,threshold=16)
# after_ip = len(input_tensor)
# after_tar = len(target_tensor)

# print('{} and {} sentences removed from ip and target tensor after filtering by threshold'.format((prev_ip-after_ip),(prev_tar-after_tar)))
# input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,#set the length of the sequences to be same by padding zeros in the end
#                                                         padding='post')
# target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,#set the length of the sequences to be same by padding zeros in the end
#                                                         padding='post')
# max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
# print(max_length_targ)
# print(max_length_inp)
# print('Maximum length of input and target tensor is {} and {}'.format(max_length_inp,max_length_targ))

In [0]:
# Try experimenting with the size of that dataset
num_examples = 50000
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

In [20]:
# Creating training and validation sets using an 99-2 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.02)
input_tensor_train,dev1_input_tensor_train,target_tensor_train,dev1_target_tensor_train=train_test_split(input_tensor_train,target_tensor_train,test_size=0.02)
input_tensor_train,dev2_input_tensor_train,target_tensor_train,dev2_target_tensor_train=train_test_split(input_tensor_train,target_tensor_train,test_size=0.02)


print('''Train set size {}, 
Test set size is {},
Dev1 set size is {},        
Dev2 set size is {}'''.format(len(input_tensor_train),len(input_tensor_val),len(dev1_input_tensor_train),len(dev2_input_tensor_train)))

Train set size 47059, 
Test set size is 1000,
Dev1 set size is 980,        
Dev2 set size is 961


In [0]:
# def convert(lang, tensor):
#   for t in tensor:
#     if t!=0:
#       print ("%d ----> %s" % (t, lang.index_word[t]))

In [0]:
# print ("Input Language; index to word mapping")
# convert(inp_lang, input_tensor_train[0])
# print ()
# print ("Target Language; index to word mapping")
# convert(targ_lang, target_tensor_train[0])

### Create a tf.data dataset

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 256
#steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)
#vocab_inp_size = len(inp_lang.word_index)+1
#vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [24]:
print(vocab_inp_size,vocab_tar_size)

12999 6817


In [25]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([Dimension(256), Dimension(16)]),
 TensorShape([Dimension(256), Dimension(12)]))

## Write the encoder and decoder model

Here, we'll implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://www.tensorflow.org/tutorials/seq2seq). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://www.tensorflow.org/tutorials/seq2seq#background_on_the_attention_mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence.

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

We're using *Bahdanau attention*. Lets decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU

The shapes of all the vectors at each step have been specified in the comments in the code:

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True,
                               return_state=True,
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru_fw = gru(self.enc_units)
    self.bi_gru=tf.keras.layers.Bidirectional(self.gru_fw,merge_mode='ave')

  def call(self, x, hidden1,hidden2):
    x = self.embedding(x)
    output, fw_state, bk_state = self.bi_gru(x, initial_state = [hidden1,hidden2])
    state = tf.math.add(fw_state,bk_state)/2    
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [28]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden1 = encoder.initialize_hidden_state()
sample_hidden2 = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden1,sample_hidden2)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (256, 16, 1024)
Encoder Hidden state shape: (batch size, units) (256, 1024)


In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, hidden_size)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [30]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (256, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (256, 16, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.dec_units)
    self.fc = tf.keras.layers.Dense(vocab_size)
    


    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [32]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, state,attention_weights = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (256, 6817)


## Define the optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='validation_accuracy')

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [0]:
def evaluate(sentence,unk_id,sample_size):
  
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence,language='english')
    inputs = [inp_lang.word_index.get(i,unk_id) for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''

    hidden1 = tf.zeros((1, units))
    hidden2 = tf.zeros((1, units))
    enc_out, enc_hidden = encoder(inputs, hidden1,hidden2)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        #predicted_id = tf.argmax(predictions[0]).numpy()
        
        #Implement top-n sampling decoding technique
        distribution = tf.argsort(predictions[0],direction='DESCENDING').numpy()[:sample_size]
        #random.seed(2)
        predicted_id = random.choice(distribution)
        
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# @tf.function
# def train_step(inp, targ, enc_hidden_fw,enc_hidden_bk):
#   loss = 0
#   acc = 0
#   with tf.GradientTape() as tape:
#     enc_output, enc_hidden = encoder(inp, enc_hidden_fw,enc_hidden_bk)

#     dec_hidden = enc_hidden

#     dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

#     # Teacher forcing - feeding the target as the next input
#     for t in range(1, targ.shape[1]):
#       # passing enc_output to the decoder
      
#       predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

#       loss += loss_function(targ[:, t], predictions)
#       acc+=train_accuracy(targ[:, t], predictions)


#       # using teacher forcing
#       dec_input = tf.expand_dims(targ[:, t], 1)
      
  
#   batch_loss = (loss / int(targ.shape[1]))
#   batch_accuracy = (acc / int(targ.shape[1]))
  
#   variables = encoder.trainable_variables + decoder.trainable_variables
  
#   gradients = tape.gradient(loss, variables)
  
#   optimizer.apply_gradients(zip(gradients, variables))
  
# #   train_loss(loss)
# #   train_accuracy(targ[:, t], predictions)
  
#   return (batch_loss,batch_accuracy)

In [0]:
def batch_translation_with_greedy(encoder,decoder):
  
  for val_ip,target in zip(dev1_input_tensor_train,dev1_target_tensor_train):
    inputs = tf.keras.preprocessing.sequence.pad_sequences([val_ip],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    hidden1 = tf.zeros((1, units))
    hidden2 = tf.zeros((1, units))
    enc_out, enc_hidden = encoder(inputs, hidden1,hidden2)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)
    
    result=[]
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(predicted_id)
      
        if predicted_id == '1':
            target = [tf.expand_dims(i, 0) for i in target]
            result = [tf.expand_dims(i, 0) for i in result]
            
            return valid_accuracy(target,result).numpy()/len(target)
            

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
    target = [tf.expand_dims(i, 0) for i in target]
    result = [tf.expand_dims(i, 0) for i in result]
    return valid_accuracy(target,result).numpy()/len(target)

In [0]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    total_acc = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        acc=0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden,hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                acc+=train_accuracy(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        batch_accuracy = (acc / int(targ.shape[1]))
        total_loss += batch_loss
        total_acc  += batch_accuracy
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f} Train_accuracy {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy(),batch_accuracy/BATCH_SIZE))
    # saving (checkpoint) the model every 2 epochs
    #if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f} Train_accuracy {:.4f}'.format(epoch + 1,total_loss/N_BATCH,total_acc / (N_BATCH*BATCH_SIZE)))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    if (epoch + 1) % 3 == 0:
      checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
      dev_Accuracy = batch_translation_with_greedy(encoder,decoder)
      print('dev set accuracy on the entire dev set 1 is {}'.format(dev_Accuracy))

W0703 05:46:17.115728 139874475698048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:454: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Epoch 1 Batch 0 Loss 4.7729 Train_accuracy 0.0000
Epoch 1 Batch 100 Loss 2.2323 Train_accuracy 0.0006
Epoch 1 Loss 434.9617 Train_accuracy 0.0005
Time taken for 1 epoch 126.1062536239624 sec

dev set accuracy on the entire dev set 1 is 0.027777778605620067
Epoch 2 Batch 0 Loss 1.8941 Train_accuracy 0.0007
Epoch 2 Batch 100 Loss 1.7521 Train_accuracy 0.0008
Epoch 2 Loss 326.2333 Train_accuracy 0.0007
Time taken for 1 epoch 126.77864170074463 sec

dev set accuracy on the entire dev set 1 is 0.027777778605620067
Epoch 3 Batch 0 Loss 1.5481 Train_accuracy 0.0008
Epoch 3 Batch 100 Loss 1.5056 Train_accuracy 0.0009
Epoch 3 Loss 276.9001 Train_accuracy 0.0009
Time taken for 1 epoch 126.92377185821533 sec

dev set accuracy on the entire dev set 1 is 0.027777778605620067
Epoch 4 Batch 0 Loss 1.5481 Train_accuracy 0.0009
Epoch 4 Batch 100 Loss 1.5056 Train_accuracy 0.0009
Epoch 4 Loss 276.9002 Train_accuracy 0.0009
Time taken for 1 epoch 128.12685680389404 sec

dev set accuracy on the entire dev

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [0]:
# def evaluate_using_bs(sentence,unk_id,beam_width):
#     units = 1024
#     beam_width = beam_width
#     result = ''
#     hidden1 = tf.zeros((1, units))
#     hidden2 = tf.zeros((1, units))
#     embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)
    
#     end_token = 2
    
#     beam_cel = tf.nn.rnn_cell.GRUCell(units)
#     sentence = preprocess_sentence(sentence,language='english')
#     inputs = [inp_lang.word_index.get(i,unk_id) for i in sentence.split(' ')]
#     inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
#                                                            maxlen=max_length_inp,
#                                                            padding='post')
#     inputs              = tf.convert_to_tensor(inputs)
#     enc_out, enc_hidden = encoder(inputs, hidden1,hidden2)
#     batch_size_beam      = tf.shape(enc_out)[0]
#     start_tokens = tf.tile(tf.constant([1], dtype=tf.int32), [ batch_size_beam//beam_width ] )
#     #print('till1')
#     enc_rnn_out_beam   = tf.contrib.seq2seq.tile_batch(enc_out, beam_width )
#     #print('till2')
#     #seq_len_beam       = tf.contrib.seq2seq.tile_batch(enc_out.shape[1], beam_width )
#     #print('till3')
#     enc_rnn_state_beam = tf.contrib.seq2seq.tile_batch(enc_hidden, beam_width )
#     #print('till3')
#     out_layer          = tf.keras.layers.Dense(vocab_tar_size)
#     #print('out')
#     attn_mech_beam = tf.contrib.seq2seq.BahdanauAttention(num_units = units,  memory = enc_out, normalize=True)
#     print('cell_imp1')
#     cell_beam = tf.contrib.seq2seq.AttentionWrapper(cell=beam_cel,attention_mechanism=attn_mech_beam,
#                                                     attention_layer_size=units)
#     print('cell_imp2')
    
#     #initial_state_beam = cell_beam.zero_state(batch_size=batch_size_beam,dtype=tf.float32).clone(cell_state=enc_rnn_state_beam)
#     initial_state_beam = cell_beam.zero_state(batch_size_beam, tf.float32)
#     print('cell_imp3')   
#     print(type(cell_beam),'cb')
#     #print((embedding.shape),'eb_out')
#     #print(tf.shape(cell_beam))
#     my_decoder = tf.contrib.seq2seq.BeamSearchDecoder(cell = cell_beam,
#                                                      embedding = embedding,
#                                                      start_tokens = start_tokens,
#                                                      end_token = end_token,
#                                                      initial_state = initial_state_beam,
#                                                      beam_width = beam_width,
#                                                      output_layer=out_layer)
#     print('hrere')
#     beam_output, t1 , t2 = tf.contrib.seq2seq.dynamic_decode(my_decoder)

#     beam_logits = tf.no_op()
#     predicted_ids = beam_output.predicted_ids

#     for predicted_id in predicted_ids:
#       result += targ_lang.index_word[predicted_id] + ' '

#       if targ_lang.index_word[predicted_id] == '<end>':
#           return result, sentence

        

#     return result, sentence

In [0]:
# beam search
def beam_search_decoder(data, k):
	sequences = [[list(), 1.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score * -log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences

In [0]:
def evaluate_using_bs1(sentence,unk_id,beam_width):
  
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence,language='english')
    inputs = [inp_lang.word_index.get(i,unk_id) for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''

    hidden1 = tf.zeros((1, units))
    hidden2 = tf.zeros((1, units))
    enc_out, enc_hidden = encoder(inputs, hidden1,hidden2)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)
    data=[]
    main=[]
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        data.append(tf.nn.softmax(predictions[0]).numpy())
        
        
        bresult = beam_search_decoder(np.array(data), beam_width)
        #print(bresult)
        main.append(bresult[0][0])
        for seq in bresult:
          for predicted_id in seq[0]:
            result += targ_lang.index_word[predicted_id] + ' '
            if targ_lang.index_word[predicted_id] == '<end>':
              return main, sentence
            dec_input = tf.expand_dims([predicted_id], 0)
    return main, sentence

In [0]:
def translate_using_bs(sentence,beam_width):
    result, sentence = evaluate_using_bs1(sentence,unk_id,beam_width)
    #print(result[-1])
    result = ' '.join([targ_lang.index_word[i] for i in result[-1]])
    return result
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [0]:
#[targ_lang.index_word[i] for i in [9, 394, 9027, 9, 9028, 1099, 2]]

In [0]:
#translate_using_bs1('you work well only with log sentences',2)

In [0]:
def evaluate(sentence,unk_id,sample_size):
  
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence,language='english')
    inputs = [inp_lang.word_index.get(i,unk_id) for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''

    hidden1 = tf.zeros((1, units))
    hidden2 = tf.zeros((1, units))
    enc_out, enc_hidden = encoder(inputs, hidden1,hidden2)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        #predicted_id = tf.argmax(predictions[0]).numpy()
        
        #Implement top-n sampling decoding technique
        distribution = tf.argsort(predictions[0],direction='DESCENDING').numpy()[:sample_size]
        #random.seed(2)
        predicted_id = random.choice(distribution)
        
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [0]:
def translate(sentence,sample_size):
    result, sentence, attention_plot = evaluate(sentence,unk_id,sample_size=sample_size)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [0]:
# restoring the latest checkpoint in checkpoint_dir

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)).assert_consumed()

In [0]:

def calc_BLEU(decoder_type='greedy'):
  start=time.time()
  score = 0
  score_2=0
  score_3=0
  score_4=0
  for targ,sour_input in zip(targ_lang.sequences_to_texts(target_tensor_val),inp_lang.sequences_to_texts(input_tensor_val)):
  
      
    sour_input = sour_input.replace('<start>','').replace('<end>','')
    targ = targ.replace('<start>','').replace('<end>','')
    
    if decoder_type == 'greedy':
      try:
        result,_,_=evaluate(sour_input,unk_id,sample_size=1)
      except KeyError:
        continue
        print(sour_input)
        
      score +=sentence_bleu([targ],result)
      
    elif decoder_type == 'sampling_dec':
      try:
        result_2,_,_=evaluate(sour_input,unk_id,sample_size=2)
        result_3,_,_=evaluate(sour_input,unk_id,sample_size=3)
        result_4,_,_=evaluate(sour_input,unk_id,sample_size=4)
      except KeyError:
        continue
        print(sour_input)
      score_2 +=sentence_bleu([targ],result_2)
      score_3 +=sentence_bleu([targ],result_3)
      score_4 +=sentence_bleu([targ],result_4)
      score = (score_2+score_3+score_4)/3
    
    elif decoder_type == 'beam_search':
      try:
        result_2=translate_using_bs(sour_input,beam_width=2)
        result_3=translate_using_bs(sour_input,beam_width=3)
        result_4=translate_using_bs(sour_input,beam_width=4)
      except KeyError:
        continue
        print(sour_input)
      score_2 +=sentence_bleu([targ],result_2)
      score_3 +=sentence_bleu([targ],result_3)
      score_4 +=sentence_bleu([targ],result_4)
      score = (score_2+score_3+score_4)/3
  print(score)        
  print('processing time for {0} decoder is {1}'.format(decoder_type,str(time.time()-start)))
  return(score/len(target_tensor_val) )

In [0]:
Bleu_score_beam_search = calc_BLEU('beam_search')

In [0]:
Bleu_score_greedy

In [0]:
Bleu_score_greedy = calc_BLEU('greedy')
print(Bleu_score_greedy)

In [0]:
random.seed(2)
n=1
translate(u'you work well only with need long sentence',n)
translate(u'do not take sufficient water',n)
translate(u'You must specify a folder',n)
translate(u'Executor of His own will',n)
translate(u'This will keep getting better',n)
translate(u'Sorry for confusing you',n)
translate(u'Hey who are you man',n)
translate(u'Message from server',n)
translate(u"I have three sons",n)
translate(u'it should be done',n)
translate(u'Are you sure you want to delete this feed?',n)
translate(u'there is a mark on the ruler',n)
translate(u'Shooting in America',n)

In [0]:
n_beams=[2,3,4]
for i in n_beams:
  n=i
  translate_using_bs('you work well only with need long sentence',n)
  translate_using_bs('do not take sufficient water',n)
  translate_using_bs('You must specify a folder',n)
  translate_using_bs(u'Executor of His own will',n)
  translate_using_bs(u'This will keep getting better',n)
  translate_using_bs(u'Sorry for confusing you',n)
  translate_using_bs(u'Hey who are you man',n)
  translate_using_bs(u'Message from server',n)
  translate_using_bs("I have three sons",n)
  translate_using_bs('it should be done',n)
  translate_using_bs('Are you sure you want to delete this feed ?',n)
  translate_using_bs('there is a mark on the ruler',n)
  translate_using_bs('Shooting in America',n)

In [0]:
Bleu_score_beam_search

In [0]:
#Baseline BLEU = 0.21320239655511655
Bleu_score_beam_search = calc_BLEU('beam_search')
Bleu_score_greedy = calc_BLEU('greedy')
Bleu_score_sampling_decoder = calc_BLEU('sampling_dec')
print(Bleu_score_beam_search)
print(Bleu_score_greedy)
print(Bleu_score_sampling_decoder)

In [0]:
import matplotlib.pyplot as plt
plt.plot(range(1,15), [6.56,5.6,5.44,5.35,4.58,3.92,3.4,2.92,2.4,2.18,1.9,1.65,1.43,1.28], 'ro')
#plt.axis([0, 6, 0, 20])
plt.show()

 ## Read this ...very important
 
 https://github.com/tensorflow/nmt#beam-search
 
 ## Word embedding reference
 
 https://github.com/monk1337/word_embedding-in-tensorflow/blob/master/Use%20Pre-trained%20word_embedding%20in%20Tensorflow.ipynb

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
